<a href="https://colab.research.google.com/github/louisowen6/NLP_Stacking_Ensemble/blob/master/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount the Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Import Packages

In [0]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import LeaveOneOut,StratifiedKFold
from sklearn.metrics.pairwise import cosine_similarity

# Run it to obtain reproducible results across machines (from keras.io)
from __future__ import print_function
import tensorflow as tf
import random as rn
import os
from tensorflow.keras import optimizers,backend as K
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input,Dense, Activation,Dropout, Embedding,Conv1D,GlobalMaxPooling1D,Concatenate,GaussianNoise,LSTM,BatchNormalization
from tensorflow.keras.utils import plot_model

# Import Data

In [0]:
link ='https://drive.google.com/open?id=1OTp9-w7uVdHJr2llOG3qZ2Rcv3W5jF50'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pred_CNN_W2V.csv')  
pred_CNN_W2V = pd.read_csv('pred_CNN_W2V.csv')
pred_CNN_W2V=pred_CNN_W2V.drop(['Unnamed: 0'],1)

link ='https://drive.google.com/open?id=1M0h0f5gl5DF6x-iboArT4CIdMnS3Wtpd'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pred_lstm_GloVe.csv')  
pred_lstm_GloVe = pd.read_csv('pred_lstm_GloVe.csv')
pred_lstm_GloVe=pred_lstm_GloVe.drop(['Unnamed: 0'],1)

link ='https://drive.google.com/open?id=1hUIsGgxjulofDekDz5kuDD89Xk7q52lV'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pred_lstm_W2V.csv')  
pred_lstm_W2V = pd.read_csv('pred_lstm_W2V.csv')
pred_lstm_W2V=pred_lstm_W2V.drop(['Unnamed: 0'],1)

link ='https://drive.google.com/open?id=1SnfLmRvEaeHFXbxJso-qv5ZnZcqeSL5Y'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pred_MLP_Feature_Driven.csv')  
pred_MLP_Feature_Driven = pd.read_csv('pred_MLP_Feature_Driven.csv')
pred_MLP_Feature_Driven=pred_MLP_Feature_Driven.drop(['Unnamed: 0'],1)

# link ='https://drive.google.com/open?id=1UaD6hNj9DajXHSBAYPsl2UGXp3g0Lu73'
# fluff, id = link.split('=')
# print (id) # Verify that you have everything after '='
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('pred_MLP_GloVe_Sentence_Vector.csv')  
# pred_MLP_GloVe_Sentence_Vector = pd.read_csv('pred_MLP_GloVe_Sentence_Vector.csv')
# pred_MLP_GloVe_Sentence_Vector=pred_MLP_GloVe_Sentence_Vector.drop(['Unnamed: 0'],1)

link ='https://drive.google.com/open?id=12f7_dOk1iMuZ8d_eseoSoblzfcZK2jMM'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('pred_MLP_W2V_Sentence_Vector.csv')  
pred_MLP_W2V_Sentence_Vector = pd.read_csv('pred_MLP_W2V_Sentence_Vector.csv')
pred_MLP_W2V_Sentence_Vector=pred_MLP_W2V_Sentence_Vector.drop(['Unnamed: 0'],1)

link ='https://drive.google.com/open?id=1cDKGfay0oLkGQM0jz4nyk6c9CoE2YIpi'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('df_test.csv')  
df_test = pd.read_csv('df_test.csv')
df_test=df_test.drop(['Unnamed: 0','index'],1)

1OTp9-w7uVdHJr2llOG3qZ2Rcv3W5jF50
1M0h0f5gl5DF6x-iboArT4CIdMnS3Wtpd
1hUIsGgxjulofDekDz5kuDD89Xk7q52lV
1SnfLmRvEaeHFXbxJso-qv5ZnZcqeSL5Y
12f7_dOk1iMuZ8d_eseoSoblzfcZK2jMM
1cDKGfay0oLkGQM0jz4nyk6c9CoE2YIpi


# Data Concatenation

In [0]:
df=pred_MLP_Feature_Driven.copy()
df['pred_MLP_W2V_Sentence_Vector']=pred_MLP_W2V_Sentence_Vector['pred_MLP_W2V_Sentence_Vector']
# df['pred_MLP_GloVe_Sentence_Vector']=pred_MLP_GloVe_Sentence_Vector['pred_MLP_GloVe_Sentence_Vector']
df['pred_lstm_W2V']=pred_lstm_W2V['pred_lstm_W2V']
df['pred_lstm_GloVe']=pred_lstm_GloVe['pred_lstm_GloVe']
df['pred_CNN_W2V']=pred_CNN_W2V['pred_CNN_W2V']
df['source']=df_test['source']
df['sentiment score']=df_test['sentiment score']

In [0]:
df.head()

,pred_MLP_Feature_Driven,pred_MLP_W2V_Sentence_Vector,pred_lstm_W2V,pred_lstm_GloVe,pred_CNN_W2V,source,sentiment score
0,0.373551,0.243758,0.442542,0.419690,0.478512,stocktwits,0.425
1,-0.408889,-0.467696,-0.482740,-0.437111,-0.501353,twitter,-0.463
2,0.374590,0.452197,0.377545,0.180178,0.204984,stocktwits,0.738
3,-0.096241,0.210885,0.469682,0.276175,0.220091,stocktwits,-0.540
4,0.173559,-0.377242,0.150323,-0.187894,0.013896,stocktwits,0.228


# MLP Ensemble

## Function Definition

In [0]:
def grid_model_ensemble(df,columns,hidden_nodes,dropout,activation_list,optimizer,epochs):
  '''
  Function for hyperparameter tuning
  '''
  kfold = StratifiedKFold(n_splits=30, shuffle=True, random_state=500)
  stratify=df['source']
  X=df[columns]
  y=df['sentiment score']

  for act in activation_list:
    cvscores=[]
    start=time()
    for train_index, val_index in kfold.split(X,stratify):
      X_train, X_val = X.loc[train_index,:], X.loc[val_index,:]
      y_train, y_val = y[train_index], y[val_index]

      #Define MLP Ensemble
      model = Sequential()
      if dropout==0:
        model.add(Dense(hidden_nodes,input_shape=(X_train.shape[1],)))
      else:
        model.add(Dropout(dropout, input_shape=(X_train.shape[1],)))
        model.add(Dense(hidden_nodes))
      model.add(Activation(act))
      model.add(Dense(1,activation='tanh'))
      model.compile(loss='MSE',optimizer=optimizer,metrics=['cosine_proximity']) 
      history=model.fit(X_train, y_train,epochs=epochs,batch_size=16,validation_data=(X_val,y_val),verbose=False)
      # plt.figure(figsize=(6,6))
      # plt.plot(history.history['loss'])
      # plt.plot(history.history['val_loss'])
      # plt.title('model loss')
      # plt.ylabel('loss')
      # plt.xlabel('epoch')
      # plt.ylim((0,0.15))
      # plt.legend(['train', 'val'], loc='upper left')
      # plt.show()
      pred_val=model.predict(X_val)
      pred_val=pred_val.reshape(1,-1)
      cos=cosine_similarity(pred_val,y_val.values.reshape(1,-1))[0][0]
      # print('Cosine Similarity: {}'.format(cos))
      # print('Pred Result: {}, Actual Value: {}'.format(pred_val[0][0],y_val.values[0]))
      cvscores.append(cos)
    end=time()
    print('')
    print('Time Elapsed: {}'.format(end-start))
    print('Epochs: {}'.format(epochs))
    print('Hidden Nodes: {}'.format(hidden_nodes))
    print('Dropout Rate: {}'.format(dropout))
    print('Activation Function: {}'.format(act))
    print('Optimizer: {}'.format(optimizer))
    print('Stratified 30-Fold Cosine Similarity Avg Score: {}, Std Score: {}'.format(np.mean(cvscores),np.std(cvscores)))
    # print('Leave One Out Cosine Similarity Score: {}'.format(cosine_similarity(pd.Series(preds).values.reshape(1,-1),y.values.reshape(1,-1))[0][0]))


## Hyperparameter Tuning

In [0]:
used_variables=['pred_MLP_Feature_Driven','pred_CNN_W2V','pred_lstm_W2V','pred_MLP_W2V_Sentence_Vector']
adam=tf.keras.optimizers.Adam(learning_rate=0.0075)
grid_model_ensemble(df,columns=used_variables,hidden_nodes=4,dropout=0.15,activation_list=['relu','tanh','elu'],optimizer=adam,epochs=25)


Time Elapsed: 196.59793400764465
Epochs: 25
Hidden Nodes: 4
Dropout Rate: 0.15
Activation Function: relu
Optimizer: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f9de0cff940>
Stratified 30-Fold Cosine Similarity Avg Score: 0.8714564951769027, Std Score: 0.09655659032466836

Time Elapsed: 264.3385579586029
Epochs: 25
Hidden Nodes: 4
Dropout Rate: 0.15
Activation Function: tanh
Optimizer: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f9de0cff940>
Stratified 30-Fold Cosine Similarity Avg Score: 0.8691887832219654, Std Score: 0.10099400506506054

Time Elapsed: 337.05430150032043
Epochs: 25
Hidden Nodes: 4
Dropout Rate: 0.15
Activation Function: elu
Optimizer: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f9de0cff940>
Stratified 30-Fold Cosine Similarity Avg Score: 0.8739162765631422, Std Score: 0.09198505267429419


In [0]:
used_variables=['pred_MLP_Feature_Driven','pred_CNN_W2V','pred_lstm_GloVe','pred_MLP_W2V_Sentence_Vector']
adam=tf.keras.optimizers.Adam(learning_rate=0.0075)
grid_model_ensemble(df,columns=used_variables,hidden_nodes=4,dropout=0.15,activation_list=['elu'],optimizer=adam,epochs=25)


Time Elapsed: 410.4883306026459
Epochs: 25
Hidden Nodes: 4
Dropout Rate: 0.15
Activation Function: elu
Optimizer: <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f9de0cff898>
Stratified 30-Fold Cosine Similarity Avg Score: 0.876555093520533, Std Score: 0.08711577892928793


# Baseline (Avg) Ensemble

In [0]:
#pred_MLP_Feature_Driven, pred_CNN_W2V, pred_MLP_W2V_Sentence_Vector, pred_lstm_W2V
pred=(df['pred_MLP_Feature_Driven']+df['pred_CNN_W2V']+df['pred_MLP_W2V_Sentence_Vector']+df['pred_lstm_W2V'])/4
print('Cosine Similarity: {}'.format(cosine_similarity(pred.values.reshape(1,-1),df['sentiment score'].values.reshape(1,-1))[0][0]))

#pred_MLP_Feature_Driven, pred_CNN_W2V, pred_MLP_W2V_Sentence_Vector, pred_lstm_GloVe
pred=(df['pred_MLP_Feature_Driven']+df['pred_CNN_W2V']+df['pred_MLP_W2V_Sentence_Vector']+df['pred_lstm_GloVe'])/4
print('Cosine Similarity: {}'.format(cosine_similarity(pred.values.reshape(1,-1),df['sentiment score'].values.reshape(1,-1))[0][0]))

Cosine Similarity: 0.8010748235996722
Cosine Similarity: 0.7972568537158483


# Plot Model

In [0]:
def plot(model):
  '''
  Function for plot model
  '''
  K.clear_session()
  if model.lower()=='ensemble':
    model=Sequential()
    model.add(Dropout(0.15, input_shape=(4,)))
    model.add(Dense(4))
    model.add(Activation('elu'))
    model.add(Dense(1,activation='tanh'))
    adam=tf.keras.optimizers.Adam(learning_rate=0.0075)
    plot_model(model, show_shapes=True,to_file='model_MLP_Ensemble.png')
    print('Done Plot Model Ensemble')

  elif model.lower()=='cnn_w2v':

    max_sequence_length=50
    embedding_layer = Embedding(30, 300, input_length=max_sequence_length, trainable=False) #30 is just a random number, because this number won't show in the plot
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    submodels=[]
    for kw in (1,2,3,4):    # kernel sizes
      embedded_sequences = embedding_layer(sequence_input)
      sub=GaussianNoise(0.01)(embedded_sequences)
      sub_submodels=[]
      for i in range(25):
        subsub=BatchNormalization()(sub)
        subsub=Conv1D(1,kw,padding='valid',activation='tanh',strides=1)(sub)
        subsub=GlobalMaxPooling1D()(subsub)
        subsubmodel=Model(sequence_input,subsub)
        sub_submodels.append(subsubmodel)
      sub_submodel_outputs=[subsubmodel.output for subsubmodel in sub_submodels]
      sub_out=Concatenate(axis=1)(sub_submodel_outputs)
      submodel=Model(sequence_input,sub_out)
      submodels.append(submodel)
    #Concatenate Multiple Filter Sizes
    submodel_outputs = [submodel.output for submodel in submodels]
    out = Concatenate(axis=1)(submodel_outputs)
    out = Dropout(0.35)(out)
    out = Dense(10)(out)
    out=Activation('tanh')(out)
    out = Dense(5)(out)
    out=Activation('tanh')(out)
    out = Dense(1)(out)
    big_model = Model(sequence_input,out)
    plot_model(big_model, show_shapes=True,to_file='model_CNN_W2V.png')
    print('Done Plot Model CNN W2V')

  elif model.lower()=='lstm_w2v':

    max_sequence_length=50
    embedding_layer = Embedding(30, 300, input_length=max_sequence_length, trainable=False) #30 is just a random number, because this number won't show in the plot
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    out=GaussianNoise(0.1)(embedded_sequences)
    out=tf.keras.layers.CuDNNLSTM(100,return_sequences=True)(out)
    out=tf.keras.layers.CuDNNLSTM(100)(out)
    out = Dropout(0.2)(out)
    out = Dense(50)(out)
    out=Activation('relu')(out)
    out = Dense(10)(out)
    out=Activation('tanh')(out)
    out = Dense(1)(out)
    model = Model(sequence_input,out)
    plot_model(model, show_shapes=True,to_file='model_LSTM_W2V.png')
    print('Done Plot Model LSTM W2V')

  elif model.lower()=='lstm_glove':

    max_sequence_length=50
    embedding_layer = Embedding(30, 200, input_length=max_sequence_length, trainable=False) #30 is just a random number, because this number won't show in the plot
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    out=GaussianNoise(0.1)(embedded_sequences)
    out=tf.keras.layers.CuDNNLSTM(100,return_sequences=True)(out)
    out=tf.keras.layers.CuDNNLSTM(100)(out)
    out = Dropout(0.2)(out)
    out = Dense(50)(out)
    out=Activation('relu')(out)
    out = Dense(10)(out)
    out=Activation('tanh')(out)
    out = Dense(1)(out)
    model = Model(sequence_input,out)
    plot_model(model, show_shapes=True,to_file='model_LSTM_GloVe.png')
    print('Done Plot Model LSTM GloVe')
  
  elif model.lower()=='feature_driven':

    model = Sequential()
    model.add(Dropout(0.15, input_shape=(72,),seed=0))
    model.add(Dense(50))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(30))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(15))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    plot_model(model, show_shapes=True,to_file='model_MLP_Feature_Driven.png')
    print('Done Plot Model MLP Feature Driven')
  
  elif model.lower()=='mlp_w2v':

    model = Sequential()
    model.add(Dropout(0.2, input_shape=(300,),seed=0))
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('tanh'))
    model.add(Dense(1))
    plot_model(model, show_shapes=True,to_file='model_MLP_W2V.png')
    print('Done Plot Model MLP W2V')

In [0]:
for model in ['feature_driven','cnn_w2v','lstm_GloVe','mlp_w2v','ensemble']:
  plot(model=model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Done Plot Model MLP Feature Driven
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.540041 to fit

Done Plot Model CNN W2V
Done Plot Model LSTM GloVe
Done Plot Model MLP W2V
Done Plot Model Ensemble
